<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/LSTM_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM Autoencoder

**lstm autoencoder to find the original circuit manifold**

This is the very first step to train the neural network model. Train the Encoder weight for extension model. At here we are going to train autoencoder to find the circuit manifold(latent space)


### Setting environment
- package install
- google drive mount
- path configuration

In [0]:
!pip install --quiet qiskit nxpd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# TODO - modify path to fit your computer
# dataset_path - where to save the dataset including circuit information
# model_path = where to save the deep learning model you created
# weight_path = Where to save the model weight you trained

dataset_path = '/content/drive/My Drive/QUANTUM/qaward/dataset'
model_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'
weight_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'

### Train step 0 - load the total dataset
- Load dataset
- Divide it to Train / Validation / Test (default 70% / 10% / 20%)
- check the shape of data

In [0]:
import os
import pickle

# TODO - Choose your dataset name
dataset_name = "5_qubit_encoded_dataset.pkl"
with open(os.path.join(dataset_path, dataset_name), 'rb') as f:
    total_dataset = pickle.load(f)

**Store dataset as a global variable**

In [0]:
for k, v in total_dataset.items():
    print(k)
    exec("{} = v".format(k))

**Devide dataset to `Train / Validation / Test` set**

    Train - 70%
    Validation - 10%
    Test - 20%

In [0]:
data_len = enc_in.shape[0]
# TODO - choose the ratio
# Test ratio will be choosed by train and validation ratio
# 0.7 means 70% if dataset is 10,000 it will be 7,000
train_ratio = 0.7
validation_ratio = 0.1

# this ratio will be used on the below code
# first ratio - fr / second ratio - sr
fr = train_ratio
sr = train_ratio + validation_ratio

# encoder input
train_enc_in = enc_in[:round(data_len*fr)]
validation_enc_in = enc_in[round(data_len*fr):round(data_len*sr)]
test_enc_in = enc_in[round(data_len*sr):]

# decode type output
train_type = enc_out_type[:round(data_len*fr)]
validation_type = enc_out_type[round(data_len*fr):round(data_len*sr)]
test_type = enc_out_type[round(data_len*sr):]

# decoder output
# gate
train_gate = enc_out_gate[:round(data_len*fr)]
validation_gate = enc_out_gate[round(data_len*fr):round(data_len*sr)]
test_gate = enc_out_gate[round(data_len*sr):]

# control
train_control = enc_out_control[:round(data_len*fr)]
validation_control = enc_out_control[round(data_len*fr):round(data_len*sr)]
test_control = enc_out_control[round(data_len*sr):]

# target
train_target = enc_out_target[:round(data_len*fr)]
validation_target = enc_out_target[round(data_len*fr):round(data_len*sr)]
test_target = enc_out_target[round(data_len*sr):]

# classic
train_classic = enc_out_classic[:round(data_len*fr)]
validation_classic = enc_out_classic[round(data_len*fr):round(data_len*sr)]
test_classic = enc_out_classic[round(data_len*sr):]

# classic
train_angle = enc_out_angle[:round(data_len*fr)]
validation_angle = enc_out_angle[round(data_len*fr):round(data_len*sr)]
test_angle = enc_out_angle[round(data_len*sr):]

In [7]:
print(train_enc_in.shape)
print(validation_enc_in.shape)
print(test_enc_in.shape)

(21000, 65, 29)
(3000, 65, 29)
(6000, 65, 29)


### Train step 1 - Create Model
- Choose model hyperparamter
- Choose the output style


    - One Concatenated output (mse)
    - type (binary crossentropy) / angle (mse)
    - each types (categorical crossentropy) / angle (mse)

- Create lstm autoencoder
- check the model

In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, LeakyReLU
from keras.layers import BatchNormalization, concatenate
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras import objectives
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [0]:
# Model hyperparmeter
# TODO - choose the latent dimension
# This indicates the circuit latent space (manifold)
latent_dim = 32

input_dim = enc_in.shape[2]
timesteps = enc_in.shape[1]

In [0]:
def lstm_ae(input_dim, timesteps, latent_dim, output_style=0):
    """
    output_style
    0 - concatenated
        (gate + control + target + classic + angle)
        [29]
    1 - type + angle
        (gate + control + target + classic) (angle)
        [26] + [3]
    2 - Seperated   
        (gate) (control) (target) (classic) (angle)
        [8] + [6] + [6] + [6] + [3]
    """
    inputs = Input(shape=(timesteps, input_dim), name='circuit_input')
    encoded = LSTM(64)(inputs)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)

    # z = Dense(latent_dim)(encoded)
    # z = LeakyReLU(alpha=0.05)(z)

    # decoded = Dense(64)(z)
    # decoded = LeakyReLU(alpha=0.05)(decoded)
    # decoded = Dense(64)(decoded)
    # decoded = LeakyReLU(alpha=0.05)(decoded)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    # 1. Gate type (8) 
    gate = Dense(8, activation='softmax', name='gate')(decoded)
    
    # 2. Qubit / classical type (18)
    control = Dense(6, activation='softmax', name='control')(decoded)
    target = Dense(6, activation='softmax', name='target')(decoded)
    classic = Dense(6, activation='softmax', name='classic')(decoded)

    # 3. Angle type (3)
    # Angle (phi, theta, gamma) choosing
    angle = BatchNormalization()(decoded)
    angle = Dense(3, name='angle')(angle)
    
    if (output_style == 0):
        ## Concatenate the total output
        # (8 + 6 + 6 + 6 + 3)
        # (29)
        total = concatenate([gate, control, target, classic, angle], name='circuit')
        lstm_autoencoder = Model(inputs, total)
        lstm_autoencoder.compile(optimizer=Adam(0.001), loss='mse', metrics=['accuracy'])
        encoder = Model(inputs, encoded)
        return (lstm_autoencoder, encoder)
    elif (output_style == 1):
        ## Concatenate the types
        # (8 + 6 + 6 + 6) + (3)
        # (26) + (3)
        total_type = concatenate([gate, control, target, classic], name='type')
        lstm_autoencoder = Model(inputs, [total_type, angle])
        lstm_autoencoder.compile(optimizer=Adam(0.001),
                                 loss=['binary_crossentropy', 'mse'],
                                 loss_weights=[0.5, 0.5],
                                 metrics=['accuracy'])
        encoder = Model(inputs, encoded)
        return (lstm_autoencoder, encoder)
    elif (output_style == 2):
        ## Seperated output
        # (8) + (6) + (6) + (6) + (3)
        lstm_autoencoder = Model(inputs, [gate, control, target, classic, angle])
        lstm_autoencoder.compile(optimizer=Adagrad(0.001),
                                loss=['categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'mse'],
                                loss_weights=[0.2, 0.2, 0.2, 0.2, 0.2],
                                metrics=['accuracy'])
        # encoder = Model(inputs, z)
        encoder = Model(inputs, encoded)
        return (lstm_autoencoder, encoder)
    else:
        raise "You have to choose output style one of between [0, 1, 2]"

In [13]:
# TODO - Choose the output style 
# Model and dataset will be set automatically depends on the output style
"""
    output_style
    0 - concatenated
        (gate + control + target + classic + angle)
        [29]
    1 - type + angle
        (gate + control + target + classic) (angle)
        [26] + [3]
    2 - Seperated   
        (gate) (control) (target) (classic) (angle)
        [8] + [6] + [6] + [6] + [3]
"""
output_style = 0

model, encoder = lstm_ae(input_dim, timesteps, latent_dim, output_style=output_style)
model.summary()
encoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
circuit_input (InputLayer)      (None, 65, 29)       0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 64)           24064       circuit_input[0][0]              
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 65, 64)       0           lstm_3[0][0]                     
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 65, 29)       10904       repeat_vector_2[0][0]            
____________________________________________________________________________________________

### Train step 2 - Train the model
- Choose train hyperparameter
    - epochs
    - batch_size
    - shuffle
- Save the model and weight
- Load the weight for sequential train

#### Load the weight for sequential train
**Model should be already pre-built**

In [0]:
# TODO - Choose weight name to load
model_weight = "lstm_ae.h5"
encoder_weight = "lstm_ae_enc.h5"

model.load_weights(os.path.join(weight_path, model_weight))
encoder.load_weights(os.path.join(weight_path, encoder_weight))
print("Loaded weight to model")

#### Train

In [0]:
# Train hyperparameter
# TODO - Choose the hyperparameter
shuffle = True
epochs = 100
batch_size = 128

if (output_style == 0):
    model.fit(train_enc_in, train_enc_in,
            verbose=1,
            shuffle=shuffle,
            epochs=epochs,
            batch_size=batch_size,  
            validation_data=(validation_enc_in, validation_enc_in))
elif (output_style == 1):
    model.fit(train_enc_in,
            [train_type,
            train_angle],
            verbose=1,
            shuffle=shuffle,
            epochs=epochs,
            batch_size=batch_size,  
            validation_data=(validation_enc_in, [validation_type,
                                                validation_angle]),)
elif (output_style == 2):
    model.fit(train_enc_in,
            [train_gate,
            train_control,
            train_target,
            train_classic,
            train_angle],
            verbose=1,
            shuffle=shuffle,
            epochs=epochs,
            batch_size=batch_size,  
            validation_data=(validation_enc_in, [validation_gate,
                                                validation_control,
                                                validation_target,
                                                validation_classic,
                                                validation_angle]),)
else:
    raise "You have to choose output style one of between [0, 1, 2]"

**Save model and weight**

In [0]:
# TODO - Choose the model & weight name
model_name = "lstm_ae.json"
encoder_name = "lstm_ae_enc.json"

model_weight = "lstm_ae.h5"
encoder_weight = "lstm_ae_enc.h5"

model_json = model.to_json()
with open(os.path.join(model_path, model_name), "w") as f:
    f.write(model_json)
encoder_json = encoder.to_json()
with open(os.path.join(model_path, encoder_name), "w") as f:
    f.write(encoder_json)
print("Saved model to disk")
model.save_weights(os.path.join(weight_path, model_weight))
encoder.save_weights(os.path.join(weight_path, encoder_weight))
print("Saved weight to disk")

Saved model to disk
Saved weight to disk


### Train step 3 - Generate the circuit and compare with the original circuit

In [0]:
# gate type encode
gate_2_id = {'U1Gate': 0, 'U2Gate':1, 'U3Gate': 2, 'CnotGate': 3, 'Measure': 4, 'IdGate': 5, 'Barrier': 6, 'X': 7}
id_2_gate = dict((v, k) for k, v in gate_2_id.items())

In [0]:
#@title Compare the original and generated circuit
def compare_results(index, original, predict):
    print("TEST SET %d" % index)
    print("=================ORIGINAL=================\n=================RESTORE==================\n")
    print("GATE NUMBER | GATE TYPE | CONTROL TARGET MEASURE | (THETA PIE GAMMA)\n")
    for i in range(len(original)):
        # predict
        p_gate = predict[i][:8]
        p_control= predict[i][8:14]
        p_target = predict[i][14:20]
        p_classic = predict[i][20:26]
        p_angle = predict[i][26:]

        p_gate = np.argmax(p_gate)
        p_control = np.argmax(p_control)
        p_target = np.argmax(p_target)
        p_classic = np.argmax(p_classic)

        # original
        o_gate = original[i][:8]
        o_control = original[i][8:14]
        o_target = original[i][14:20]
        o_classic = original[i][20:26]
        o_angle = original[i][26:]

        o_gate = np.argmax(o_gate)
        o_control = np.argmax(o_control)
        o_target = np.argmax(o_target)
        o_classic = np.argmax(o_classic)
        
        print("%-10s" % ("GATE : {}".format(i)), end="")
        print("%-8s" % id_2_gate[o_gate], end=" ")
        print("%-4s" % "[C:{}".format(o_control - 1 if o_control else ' '), end=" ")
        print("T:{}".format(o_target - 1 if o_target else ' '), end=" ")
        print("M:{}]".format(o_classic - 1 if o_classic else ' '), end=" ")
        print("(%.2f, %.2f, %.2f)" % (o_angle[0], o_angle[1], o_angle[2]))

        print("%-10s" % " ", end="")
        print("%-8s" % id_2_gate[p_gate], end=" ")
        print("%-4s" % "[C:{}".format(p_control - 1 if p_control else ' '), end=" ")
        print("T:{}".format(p_target - 1 if p_target else ' '), end=" ")
        print("M:{}]".format(p_classic - 1 if p_classic else ' '), end=" ")
        print("(%.2f, %.2f, %.2f)" % (p_angle[0], p_angle[1], p_angle[2]), end='\n\n')

In [0]:
#@title predict circuit
def predict_circuit(index, output_style=0):
    original = test_enc_in[index]
    if (output_style == 0):
        predict = model.predict([[original]])
        predict = predict[0]
    elif (output_style == 1):
        types, angles = model.predict([[original]])
        predict = np.hstack((types[0], angles[0]))
    elif (output_style == 2):
        gates, controls, targets, classics, angles = model.predict([[original]])
        predict = np.hstack((gates[0], controls[0], targets[0], classics[0], angles[0]))
    else:
        raise "You have to choose output style one of between [0, 1, 2]"
    compare_results(index, original, predict)

In [0]:
# TODO - choose what test case you want to check

test_index = 150
predict_circuit(test_index, output_style)

TEST SET 150
=================ORIGINAL=================
=================RESTORE==================

GATE NUMBER | GATE TYPE | CONTROL TARGET MEASURE | (THETA PIE GAMMA)

GATE : 0  U2Gate   [C:1 T:  M: ] (1.57, 0.00, 1.00)
          U1Gate   [C:0 T:  M: ] (0.43, 0.05, 0.43)

GATE : 1  CnotGate [C:0 T:1 M: ] (0.00, 0.00, 0.00)
          U1Gate   [C:1 T:  M: ] (0.38, -0.02, 0.37)

GATE : 2  U2Gate   [C:1 T:  M: ] (1.57, 0.00, 1.00)
          U1Gate   [C:1 T:  M: ] (0.30, -0.01, 0.27)

GATE : 3  U1Gate   [C:2 T:  M: ] (0.00, 0.00, 0.50)
          U1Gate   [C:2 T:  M: ] (0.28, 0.02, 0.25)

GATE : 4  U2Gate   [C:2 T:  M: ] (1.57, 0.00, 1.00)
          U1Gate   [C:3 T:  M: ] (0.26, 0.04, 0.21)

GATE : 5  U1Gate   [C:2 T:  M: ] (0.00, 0.00, 0.25)
          CnotGate [C:3 T:  M: ] (0.25, 0.03, 0.18)

GATE : 6  CnotGate [C:3 T:2 M: ] (0.00, 0.00, 0.00)
          CnotGate [C:3 T:  M: ] (0.26, 0.03, 0.17)

GATE : 7  U1Gate   [C:2 T:  M: ] (0.00, 0.00, -0.25)
          CnotGate [C:4 T:  M: ] (0.27, 